In [1]:
print("OK")

OK


In [6]:
%pip install langchain
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
%pip install pinecone
%pip install pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [7]:
PINECONE_API_KEY = "62738a6a-b0f2-45df-97cb-4f55405612c3"
PINECONE_API_ENV = "gcp-starter"

In [8]:
#Extract the data from pdf

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documnts = loader.load()
    
    return documnts

In [10]:
%pip install pypdf
extracted_data = load_pdf("data/")

   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   -- ------------------------------------ 20.5/290.4 kB 682.7 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/290.4 kB 825.8 kB/s eta 0:00:01
   ----------------------------------- ---- 256.0/290.4 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 290.4/290.4 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# xtracted_data

In [14]:
# Create text chunks

def text_split(extracted_datadata):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print("Length of my chunks:", len(text_chunks))

Length of my chunks: 7020


In [20]:
# Download Embedding Model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    return embeddings

In [22]:
%pip install sentence-transformers
embeddings = download_hugging_face_embeddings()

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------- ---------------------------- 30.7/134.8 kB 435.7 kB/s eta 0:00:01
     ------------------------- ----------- 92.2/134.8 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 1.1 MB/s eta 0:00:00
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     ---------------------------------------- 59.0/59.0 kB 3.0 MB/s eta 0:00:00
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached pillow-10.3.0-cp38-cp38-win_amd64.whl.metadata (9.4 kB)
  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)

c:\Users\2009s\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [25]:
# Testin the embedding model if it is working (I mean converting text to numbers)

query_result = embeddings.embed_query("Hello World")
print("Length of the query result:", len(query_result))

print("Query result:", query_result)

Length of the query result: 384
Query result: [-0.034477315843105316, 0.03102317452430725, 0.006734929047524929, 0.026108937337994576, -0.03936201333999634, -0.16030247509479523, 0.06692397594451904, -0.006441450212150812, -0.047450464218854904, 0.014758866280317307, 0.07087530195713043, 0.055527571588754654, 0.019193321466445923, -0.02625134214758873, -0.010109540075063705, -0.026940450072288513, 0.022307438775897026, -0.022226622328162193, -0.1496925801038742, -0.017493009567260742, 0.007676264736801386, 0.05435226112604141, 0.0032544350251555443, 0.0317259319126606, -0.08462139964103699, -0.029405999928712845, 0.05159561708569527, 0.0481240451335907, -0.0033148531801998615, -0.05827915295958519, 0.04196926951408386, 0.022210637107491493, 0.12818880379199982, -0.022338969632983208, -0.011656252667307854, 0.06292839348316193, -0.032876309007406235, -0.09122602641582489, -0.031175294890999794, 0.052699558436870575, 0.04703482240438461, -0.0842030942440033, -0.030056163668632507, -0.020

In [2]:
%pip install pinecone


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [3]:
# Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

index_name = "medical-chatbot"

# Creating Embeddings for Each of The Text Chunks & storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


NameError: name 'pinecone' is not defined